### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Kidney_Chromophobe/GSE15641'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
sample_characteristics = {
    0: ['tissue: Kidney'],
    1: ['Kidney_Chromophobe: Yes'],
    2: ['age: 45'],
    3: ['gender: Male']
}

background_info_summary = "!Series_summary\t\"In order to address the progression, metastasis, and clinical heterogeneity of renal cell cancer (RCC), transcriptional profiling with oligonucleotide microarrays (22,283 genes) was done on 49 RCC tumors, 20 non-RCC renal tumors, and 23 normal kidney samples."

if "transcriptional profiling" in background_info_summary and "genes" in background_info_summary:
    is_gene_available = True

# Data availability and type conversion functions
def get_value(cell_value):
    """Extract the value after the colon in a cell value."""
    return cell_value.split(":")[-1].strip() if ":" in cell_value else None

# Placeholder for actual data existence check
for key, values in sample_characteristics.items():
    if 'Kidney_Chromophobe' in values[0]:
        trait_row = key
        def convert_trait(value):
            return 1 if 'yes' in get_value(value).lower() else 0

    if 'age' in values[0]:
        age_row = key
        def convert_age(value):
            try:
                age_value = int(get_value(value))
                return age_value if age_value >= 0 else None
            except ValueError:
                return None

    if 'gender' in values[0]:
        gender_row = key
        def convert_gender(value):
            gender_value = get_value(value).lower()
            if gender_value == 'male':
                return 1
            elif gender_value == 'female':
                return 0
            else:
                return None

# Save metadata
save_cohort_info('GSE15641', './preprocessed/Kidney_Chromophobe/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Kidney_Chromophobe', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Kidney_Chromophobe/trait_data/GSE15641.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Define identifier_key and gene_symbol_key
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Kidney_Chromophobe/gene_data/GSE15641.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Kidney_Chromophobe')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE15641', './preprocessed/Kidney_Chromophobe/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Kidney_Chromophobe/GSE15641.csv'
    unbiased_merged_data.to_csv(csv_path)
